
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/result__1_.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

_c0,_c1,_c2,_c3,_c4,_c5
id,name,age,department,salary,join_date
1,John,34,IT,75000,2015-06-01
2,Sara,28,HR,58000,2019-09-15
3,Michael,45,Finance,120000,2010-01-10
4,Karen,29,IT,70000,2020-02-19
5,David,38,Finance,90000,2017-08-23
6,Linda,33,HR,60000,2018-12-05
7,James,41,IT,110000,2013-04-15
8,Emily,27,HR,52000,2021-06-20
9,Robert,36,Finance,105000,2016-11-30


In [0]:
# Create a view or table

temp_table_name = "result__1__csv"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `result__1__csv`

_c0,_c1,_c2,_c3,_c4,_c5
id,name,age,department,salary,join_date
1,John,34,IT,75000,2015-06-01
2,Sara,28,HR,58000,2019-09-15
3,Michael,45,Finance,120000,2010-01-10
4,Karen,29,IT,70000,2020-02-19
5,David,38,Finance,90000,2017-08-23
6,Linda,33,HR,60000,2018-12-05
7,James,41,IT,110000,2013-04-15
8,Emily,27,HR,52000,2021-06-20
9,Robert,36,Finance,105000,2016-11-30


In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "result__1__csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

my_schema = StructType([
    StructField("id",IntegerType(),True),
    StructField("name",StringType(),True),
    StructField("age",IntegerType(),True),
    StructField("department",StringType(),True),
    StructField("salary",IntegerType(),True),
    StructField("join_date",StringType(),True)
])

df=spark.read.format("csv").schema(my_schema).option("header",True).load('/FileStore/tables/result__1_.csv')

df.display()
df.printSchema()

id,name,age,department,salary,join_date
1,John,34,IT,75000,2015-06-01
2,Sara,28,HR,58000,2019-09-15
3,Michael,45,Finance,120000,2010-01-10
4,Karen,29,IT,70000,2020-02-19
5,David,38,Finance,90000,2017-08-23
6,Linda,33,HR,60000,2018-12-05
7,James,41,IT,110000,2013-04-15
8,Emily,27,HR,52000,2021-06-20
9,Robert,36,Finance,105000,2016-11-30


root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- join_date: string (nullable = true)



Display records of employees aged above 30.

In [0]:

from pyspark.sql.functions import col

df.filter(col("age")>30).display()

id,name,age,department,salary,join_date
1,John,34,IT,75000,2015-06-01
3,Michael,45,Finance,120000,2010-01-10
5,David,38,Finance,90000,2017-08-23
6,Linda,33,HR,60000,2018-12-05
7,James,41,IT,110000,2013-04-15
9,Robert,36,Finance,105000,2016-11-30


Find the average salary of employees in each department.

In [0]:
from pyspark.sql.functions import *

df.groupBy("department").agg(avg("salary").alias("Averge_Salary")).display()

department,Averge_Salary
HR,56666.666666666664
Finance,105000.0
IT,85000.0


Add a column experience indicating the number of years an employee has been working.

In [0]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, year

# Calculate current year
current_year = datetime.now().year

# Add 'experience' column
df_with_experience = df.withColumn(
    "experience", lit(current_year) - year(col("join_date"))
)

# Show result
df_with_experience.display()

id,name,age,department,salary,join_date,experience
1,John,34,IT,75000,2015-06-01,9
2,Sara,28,HR,58000,2019-09-15,5
3,Michael,45,Finance,120000,2010-01-10,14
4,Karen,29,IT,70000,2020-02-19,4
5,David,38,Finance,90000,2017-08-23,7
6,Linda,33,HR,60000,2018-12-05,6
7,James,41,IT,110000,2013-04-15,11
8,Emily,27,HR,52000,2021-06-20,3
9,Robert,36,Finance,105000,2016-11-30,8


Find the top 3 highest-paid employees.

In [0]:
from pyspark.sql.functions import col, max

# Group by "name" and calculate the highest salary, then sort and limit
df.groupBy("name").agg(max("salary").alias("Highest_Salary")).orderBy(col("Highest_Salary").desc()).limit(3).display()

name,Highest_Salary
Michael,120000
James,110000
Robert,105000


Identify the department with the highest total salary.

In [0]:
from pyspark.sql.functions import col, sum as _sum

# Group by "department", calculate the total salary, and sort in descending order
department_with_highest_salary = df.groupBy("department").agg(_sum("salary").alias("total_salary")).orderBy(col("total_salary").desc()).limit(1)

# Display the result
department_with_highest_salary.display()

department,total_salary
Finance,315000


Create a new DataFrame with employees earning more than the average salary.

In [0]:
from pyspark.sql.functions import col, avg

average_salary = df.agg(avg("salary").alias("average_salary")).collect()[0]["average_salary"]

employees_above_avg_salary = df.filter(col("salary") > average_salary)

employees_above_avg_salary.display()

id,name,age,department,salary,join_date
3,Michael,45,Finance,120000,2010-01-10
5,David,38,Finance,90000,2017-08-23
7,James,41,IT,110000,2013-04-15
9,Robert,36,Finance,105000,2016-11-30


Rename the column name to employee_name.

In [0]:
df_with_renamed_column = df.withColumnRenamed("name", "employee_name")
df_with_renamed_column.display()

id,employee_name,age,department,salary,join_date
1,John,34,IT,75000,2015-06-01
2,Sara,28,HR,58000,2019-09-15
3,Michael,45,Finance,120000,2010-01-10
4,Karen,29,IT,70000,2020-02-19
5,David,38,Finance,90000,2017-08-23
6,Linda,33,HR,60000,2018-12-05
7,James,41,IT,110000,2013-04-15
8,Emily,27,HR,52000,2021-06-20
9,Robert,36,Finance,105000,2016-11-30


Find the number of employees in each department.

In [0]:
df.groupBy("department").agg(count("*").alias("employee_count")).display()

department,employee_count
HR,3
Finance,3
IT,3


Select and display only the id and name columns.

In [0]:
df.select("id", "name").display()

id,name
1,John
2,Sara
3,Michael
4,Karen
5,David
6,Linda
7,James
8,Emily
9,Robert


Check for any null values in the dataset.

In [0]:
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).display()

id,name,age,department,salary,join_date
0,0,0,0,0,0
